(1) Import Libraries and define constants

In [80]:
import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
from scipy import ndimage
import sys
import math
from PIL import Image

In [106]:
# Global Constants
side_length = 64
image_path = "./Logistic_Regression_Data/"
test_ratio = .3
epoch = 7000
learning_rate = .005
max_num_images = -1

(2) Create train and test examples:

In [82]:
def getImageArrays(path, side_length): #returns list of images arrays for a specified path
    image_names = os.listdir(path)
    image_names = image_names[:max_num_images]
    examples = []
    for image_name in image_names:
        if image_name.split(".")[-1] != "DS_Store":
            try:
                cur_image_path = path + image_name
                cur_image = np.array(ndimage.imread(cur_image_path,flatten=False))
                cur_array_resized = scipy.misc.imresize(cur_image,size=(side_length,side_length))

    #             img = Image.fromarray(cur_array_resized, 'RGB')
    #             img.save('my.png')
    #             img.show()
                cur_array_flattened = cur_array_resized.reshape((side_length*side_length*3)).T
                #print("new", cur_array_resized.shape)
                #cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
                #cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
                #print("old", cur_array_resized.shape)
                examples += [cur_array_flattened] 
            except ValueError:
                print("Error in creating examples",image_name)
                
    return examples

In [83]:
#create examples & labels
cow_images_path = image_path + "cows/"
notCow_image_path = image_path + "notcows/"

examples_cow = getImageArrays(cow_images_path, side_length)
labels_cow = np.ones(len(examples_cow))
examples_notCow = getImageArrays(notCow_image_path, side_length)
labels_notCow = np.zeros(len(examples_notCow))

examples_cow = np.array(examples_cow)
examples_notCow = np.array(examples_notCow)
#assert(examples_cow.shape[1:] == (side_length,side_length,3)), "examples_cow are invalid shape"
examples = np.concatenate((examples_cow,examples_notCow))
labels = np.concatenate((labels_cow,labels_notCow))
print("Number of Examples:",len(examples))
print("Number of Labels:",len(labels))


Error in creating examples notcow197.jpg
Error in creating examples notcow20.jpg
Error in creating examples notcow38.jpg
Error in creating examples notcow415.jpg
Number of Examples: 994
Number of Labels: 994


In [84]:
#shuffle examples and labels
shuffled_indexing = np.random.permutation(labels.shape[0])
examples = examples[shuffled_indexing]
labels = labels[shuffled_indexing]

In [85]:
#seperate train and test examples
number_examples_test = int(len(examples)*test_ratio)
number_labels_test = int(len(labels)*test_ratio)

examples_test = examples[:number_examples_test]
examples_train = examples[number_examples_test:]
labels_test = labels[:number_labels_test]
labels_train = labels[number_labels_test:]
print("Number of training examples: ", examples_train.T.shape)
print("Number of test examples: ", examples_test.T.shape)
print(labels_train.shape)

Number of training examples:  (12288, 696)
Number of test examples:  (12288, 298)
(696,)


In [86]:
#reshape labels for future matrix operations
labels_train = np.reshape(labels_train,(1,len(labels_train)))
labels_test = np.reshape(labels_test,(1,len(labels_test)))

In [87]:
#flatten examples
# flattened_train_examples = examples_train.reshape(examples_train.shape[0], -1).T
# flattened_test_examples = examples_test.reshape(examples_test.shape[0], -1).T  
# print("flattened examples",flattened_test_examples.shape,flattened_train_examples.shape)
flattened_train_examples = examples_train.T
flattened_test_examples = examples_test.T

In [88]:
# Standardize color values of the image (decrease computational cost durring cross entropy)
standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
standardized_test_examples = flattened_test_examples/255
print("standardized",standardized_test_examples.shape,standardized_train_examples.shape)

standardized (12288, 298) (12288, 696)


In [89]:
#image = np.array(ndimage.imread("../Desktop/chicken.jpeg",flatten=False))
#resizedImage = scipy.misc.imresize(image,size=(side_length,side_length)).reshape((1,side_length*side_length*3)).T

(3) Defeine basic functions

In [90]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    return output

In [91]:
def initialize_zeros(dimension): #dimension = len(flattened_examples)
    zeroed_weights = np.zeros((dimension, 1)) 
    zeroed_baisies = 0. #baisies always start at 0 b/c they are developed as the model trains
    assert(isinstance(zeroed_baisies, float) or isinstance(zeroed_baisies, int))
    
    return zeroed_weights, zeroed_baisies

In [92]:
def crossEntropyLoss(a,Y):
    m = Y.shape[1] #len(Y) doesn't work, need 2nd dimesnion
#     print(a.shape)
    #print("BEFORE:")
    #print(a)
    a[0][a[0]==1] = .999
    a[0][a[0]==0] = .001
#     a = [0.0001 for prediction in a[0] if prediction == 0]
#     a = [0.9999 for prediction in a if prediction == 1]
    #print("AFTER:")
    #print(a)
    #a = np.reshape(a,(1,a.shape[1]))
    #print(a.shape,"changed")
    #a = 0.0001 if a == 0 else a
    #a = 0.9999 if a == 1 else a
    loss = -(1.0 / m) * np.sum(Y * np.log(a) + (1.0 - Y) * np.log(1.0 - a))
#     loss = (- 1.0 / m) * np.sum(Y * np.log(a) + (1.0 - Y) * (np.log(1.0 - a)))
    return loss

(4) Foward Pass

In [93]:
def propagate(X,Y,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
#     print("propagate shapes")
#     print("X",np.shape(X))
#     print("Y",np.shape(Y))
#     print("w",np.shape(w))
#     print("b",np.shape(b))
    
    w_transposed = w.T
    num_of_examples = np.shape(X)[1]
    
    dot_product = np.dot(w_transposed,X) #dot product of X and w be careful to use correct order
    z = dot_product + b
    a = sigmoid(z) #in begining should be 0
    loss = crossEntropyLoss(a,Y)
    # dz = a-y 
    #z = wx + b 
    #1/m becuase loss uses 1/m 
    dw = (1.0/num_of_examples) * np.dot(X,(a-Y).T) 
    db = (1.0/num_of_examples) * np.sum(a-Y) #np.sum collapes input
    loss = np.squeeze(loss)
    grads = {"dw":dw,"db":db}
    return grads,loss    


In [94]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(X, Y,w,b)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[ 0.99845601]
 [ 2.39507239]]
db = 0.00145557813678
cost = 5.80154531939


(5) Optimize

In [95]:
def optimizer(w, b, X, Y, learning_rate, epoch, print_results=True):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
#     print("optimizer shapes")
#     print("X",np.shape(X))
#     print("Y",np.shape(Y))
#     print("w",np.shape(w))
#     print("b",np.shape(b))
    sample_costs = []
    costs = []
    for i in range(epoch):
        grads, cost = propagate(X,Y,w,b)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        costs += [cost]
        if print_results:
            optimizerPrinter(costs,epoch)
        if (i % 10 == 0):
            sample_costs += [cost]
#             print("cost",cost,(i+1)*100/epoch,"%")
            
    grads = {"dw":dw,"db":db}
    params = {"w":w,"b":b}
    return grads,params,costs

In [96]:
def optimizerPrinter(costs,epoch):
    starting_costs = costs[:int(epoch * .09)] #gets the first 9 percent of costs, used as a baseline
    recent_costs = costs[-10:] #get the most recent 10 costs
    avg_recent_costs = sum(recent_costs)/len(recent_costs)
    avg_starting_costs = sum(starting_costs)/len(starting_costs)
    percent = ((len(costs))/epoch) * 100
#     print("cost len:",costs,"percent:", percent,"remainder:",percent % (epoch/10))
    if (len(costs) % int(epoch/(10)) == 0.0) and (len(costs) != 1):
        percent = math.ceil(percent * .1) * 10
    cur_cost = costs[-1]
    if cur_cost > 999: #only display three place values
        cur_cost = 999
    if avg_starting_costs > 999:
        avg_starting_costs = 999
    if avg_recent_costs > 999:
        avg_recent_costs = 999
    sys.stdout.write('\r')
    sys.stdout.write(
        str(int(percent)).zfill(2) + "% " +
        "Cur Cost: " + str(cur_cost).zfill(3) +
        " Recent Costs: " + str(avg_recent_costs).zfill(3) + 
        " Starting Costs: " + str(avg_starting_costs).zfill(3))
    sys.stdout.flush()
    
    if (len(costs) % int(epoch/(10)) == 0.0) and (len(costs) != 1):
        print("")
    

In [97]:
# w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
# grads,params,cost = optimizer(w,b,X, Y, .009, 143, print_results=True)
# print ("w = " + str(params["w"]))
# print ("b = " + str(params["b"]))
# print ("dw = " + str(grads["dw"]))
# print ("db = " + str(grads["db"]))

(6) Validate

In [98]:
def predictor(X,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
#     print("predictor shapes")
#     print("X",np.shape(X))
#     print("w",np.shape(w))
#     print("b",np.shape(b))
    w_transposed = w.T
    prediction = (np.dot(w_transposed,X)) + b
    prediction = sigmoid(prediction)
    prediction *= 2.0 # so prediction is either 1 or -1
    prediction = np.floor(prediction)
    return prediction

In [99]:
w = np.array([[0.1124], [0.231]])
b = -.3
X = np.array([[1,-1.1,-3.2],[1.2,2,.1]])
print(predictor(X,w,b))

[[ 1.  1.  0.]]


(7) Driver

In [100]:
def logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch):
    # train_x = (image_width * image_width * 3, training_examples)
    # train_y = (1, train_examples)
    # test_x = (____, test_examples)
    # test_y = (1, test_examples)
    print("Logistic Regression Started :0")
    
    assert(train_X.shape[1] == train_Y.shape[1]), "train exampes dimensions invalid: Train X " + train_X.shape + " Train Y " + train_Y.shape
#     print("logisticShapes")
#     print("train x",train_X.shape)
#     print("train y",train_Y.shape)
#     print("test X",test_X.shape)
#     print("test Y",test_Y.shape)
    #Inialize w/b
    w,b = initialize_zeros(side_length*side_length*3)
    
    #Train
    grads, params, costs = optimizer(w, b, train_X, train_Y, learning_rate, epoch)
    w, b = params["w"], params["b"]

    #Eval
    prediction_train = predictor(train_X,w,b)
    accuracy_train = np.mean(np.abs(prediction_train - train_Y))
    print(accuracy_train, "train accuracy")
    accuracy_output = "train accuracy {} %" .format(100 - (accuracy_train * 100))
    print(accuracy_output)
    
    #Test
    prediction_test = predictor(test_X,w,b)
    accuracy_test = np.mean(np.abs(prediction_test - test_Y))
    print(accuracy_test, "test accuarcy")
    accuracy_output = "Actural accuarcy {} %" .format(100 - (accuracy_test * 100))
    print(accuracy_output)
    
    d = {
        "costs": costs,
        "Y_prediction_test" : prediction_test,
        "X_prediction_train" : prediction_train,
        "w" : w,
        "b" : b,
        "learning rate" : learning_rate,
        "epoch" : epoch
    }
    
    return d
        

In [107]:
train_X = standardized_train_examples
train_Y = labels_train
test_X = standardized_test_examples
test_Y = labels_test

# print(test_X.shape)
# print(test_X[0])
# print(test_X[0].shape)

# print("start")
print(train_X.shape,"train_X")
print(train_Y.shape,"train_Y")
print(test_X.shape,"test_X")
print(test_Y.shape,"test_Y")

d = logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch)
print("Complete :)") 

(12288, 696) train_X
(1, 696) train_Y
(12288, 298) test_X
(1, 298) test_Y
Logistic Regression Started :0
10% Cur Cost: 0.378953842742 Recent Costs: 0.379438427677 Starting Costs: 0.520533090086
20% Cur Cost: 0.32182524211 Recent Costs: 0.322113012691 Starting Costs: 0.520533090088
30% Cur Cost: 0.284159640229 Recent Costs: 0.284365899284 Starting Costs: 0.52053309008
40% Cur Cost: 0.255973364106 Recent Costs: 0.256133584377 Starting Costs: 0.52053309008
50% Cur Cost: 0.233554421551 Recent Costs: 0.233684598945 Starting Costs: 0.52053309008
60% Cur Cost: 0.215061881148 Recent Costs: 0.215170767193 Starting Costs: 0.52053309008
70% Cur Cost: 0.199428620009 Recent Costs: 0.19952159095 Starting Costs: 0.520533090088
80% Cur Cost: 0.185974183012 Recent Costs: 0.186054796957 Starting Costs: 0.52053309008
90% Cur Cost: 0.174236377069 Recent Costs: 0.174307115038 Starting Costs: 0.52053309008
100% Cur Cost: 0.163886423533 Recent Costs: 0.163949086931 Starting Costs: 0.52053309008
0.02442528735